
# Sistema de Recomendación

<p style="text-align: justify;">
Se presenta un sistema de recomendación de productos basado en similitud de contenido para el dataset de productos de Amazon. Se utilizan técnicas de procesamiento de texto, codificación categórica y normalización de variables numéricas para construir un modelo eficiente que sugiera productos similares a los seleccionados por un usuario. La evaluación del sistema se realiza mediante métricas de precisión, recall y F1-score.
</p>

### ¿Qué es un sistema de recomendación?

Los sistemas de recomendación son herramientas fundamentales en el comercio electrónico, permitiendo personalizar la experiencia del usuario al sugerir productos que pueden ser de su interés. En este trabajo, se implementa un sistema de recomendación basado en contenido para un conjunto de datos de productos de Amazon, utilizando técnicas de minería de texto y aprendizaje automático.

### Descripción del Dataset

El dataset utilizado proviene de Kaggle y contiene información sobre productos de Amazon, incluyendo atributos como:

- **name**: El nombre del producto
- **main_category**: La categoría principal a la que pertenece el producto
- **sub_category**: La categoría principal a la que pertenece el producto
- **image**: La imagen del producto
- **link**: El enlace de referencia del producto al sitio web de Amazon
- **ratings**: Las calificaciones otorgadas por los clientes de Amazon al producto
- **no of ratings**: La cantidad de calificaciones otorgadas a este producto en las compras de Amazon
- **discount_price**: Los precios con descuento del producto
- **actual_price**: El precio de venta al público real del producto

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/work/T-shirts and Polos.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### **Importar Librerías**

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack, csr_matrix

In [3]:
dataset = pd.read_csv("/work/T-shirts and Polos.csv")

# copia del DataFrame a trabajar
df = dataset.copy()

print(df.head(10))

                                                name   main_category  \
0                 Allen Solly Men's Regular Fit Polo  men's clothing   
1            Van Heusen Men's Regular Fit Polo Shirt  men's clothing   
2  Scott International Men's Regular Fit T-Shirt ...  men's clothing   
3  TOPLOT T-Shirt for Men||T Shirt||White T-Shirt...  men's clothing   
4     Amazon Brand - Symbol Men's Regular Polo Shirt  men's clothing   
5  LEOTUDE Women's Cottonblend Half Sleeve Oversi...  men's clothing   
6                        Chromozome Men Mens Tshirts  men's clothing   
7  London Hills Solid Men Half Sleeve Pure Cotton...  men's clothing   
8    Amazon Brand - Symbol Men's Regular Fit T-Shirt  men's clothing   
9                    Adidas Men's Regular Polo Shirt  men's clothing   

       sub_category                                              image  \
0  T-shirts & Polos  https://m.media-amazon.com/images/I/71eUwDk8z+...   
1  T-shirts & Polos  https://m.media-amazon.com/images/I/71

## Metodología

El sistema de recomendación se implementa en varias etapas:

### Preprocesamiento de Datos

1. **Eliminación de valores nulos**: Se eliminan registros con información incompleta para evitar sesgos en el modelo.

In [4]:
df = df.dropna()

2. **Procesamiento de texto mediante TF-IDF**: Se aplica el algoritmo **TF-IDF (Term Frequency - Inverse Document Frequency)** para representar los nombres y categorías de los productos, destacando términos importantes en la descripción.

In [5]:
# 1. Procesar texto con TF-IDF
text_features = df['name'].fillna('') + " " + df['main_category'].fillna('') + " " + df['sub_category'].fillna('')
tfidf = TfidfVectorizer(stop_words='english', max_features=1000)  # Reducir a 1000 características para optimizar
tfidf_matrix = tfidf.fit_transform(text_features)

3. **Codificación de variables categóricas mediante One-Hot Encoding**: Se convierten las categorías de productos en representaciones numéricas utilizando One-Hot Encoding para mejorar la representación de los datos.

In [6]:
# 2. Convertir categorías en One-Hot Encoding
ohe = OneHotEncoder(handle_unknown='ignore', sparse=True)  # Mantener en formato disperso
categories_matrix = ohe.fit_transform(df[['main_category', 'sub_category']])
categories_matrix

<12092x2 sparse matrix of type '<class 'numpy.float64'>'
	with 24184 stored elements in Compressed Sparse Row format>

In [7]:
# Limpiar valores en 'no_of_ratings'
df['no_of_ratings'] = df['no_of_ratings'].astype(str)
df['no_of_ratings'] = df['no_of_ratings'].str.replace(',', '', regex=True)
df['no_of_ratings'] = df['no_of_ratings'].str.extract('(\d+)')  # Extraer solo los números

df['actual_price'] = df['actual_price'].str.replace(',', '', regex=True).str.replace('₹', '', regex=True)
df['discount_price'] = df['discount_price'].str.replace(',', '', regex=True).str.replace('₹', '', regex=True)

# Convertir a float, reemplazando NaN con 0 si es necesario
df['no_of_ratings'] = pd.to_numeric(df['no_of_ratings'], errors='coerce').fillna(0)
df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce').fillna(0)

4. **Normalización de variables numéricas con MinMaxScaler**: Se normalizan las características como precios y calificaciones en un rango de [0,1] usando MinMaxScaler para evitar que variables con valores grandes dominen la similitud

In [8]:
# 3. Normalizar valores numéricos
numeric_features = ['ratings', 'no_of_ratings', 'actual_price']
scaler = MinMaxScaler()
numeric_matrix = scaler.fit_transform(df[numeric_features])
numeric_matrix = csr_matrix(numeric_matrix)  # Convertir en sparse matrix

5. **Reducción de dimensionalidad con Truncated SVD**: Se usa Truncated SVD para reducir la dimensión de la matriz de características textuales y mejorar la eficiencia del modelo.

In [9]:
# 4. Reducir la Dimensión del TF-IDF
svd = TruncatedSVD(n_components=50)  # Reduce a 50 componentes
tfidf_reduced = svd.fit_transform(tfidf_matrix)
tfidf_reduced = csr_matrix(tfidf_reduced)  # Convertir a sparse matrix


In [10]:
# 5. Concatenar todas las representaciones en formato disperso
final_matrix = hstack([tfidf_reduced, categories_matrix, numeric_matrix])
print(final_matrix)

  (0, 0)	0.424120549180051
  (0, 1)	-0.19936593020932108
  (0, 2)	-0.02408199883183356
  (0, 3)	-0.18038394286728443
  (0, 4)	0.0037819459414465554
  (0, 5)	0.016692995652933463
  (0, 6)	0.006556311442078637
  (0, 7)	0.027616063620166366
  (0, 8)	-0.026019079813249493
  (0, 9)	0.04550998480512318
  (0, 10)	0.019271532718903403
  (0, 11)	0.13980696196793088
  (0, 12)	-0.03080894863769102
  (0, 13)	0.11882654257501549
  (0, 14)	0.06360786239383547
  (0, 15)	-0.06455937142065363
  (0, 16)	0.13993513324937176
  (0, 17)	0.30625901427399316
  (0, 18)	0.05394359857056824
  (0, 19)	0.6624142800495754
  (0, 20)	-0.305859447980003
  (0, 21)	-0.07991128486998997
  (0, 22)	0.00013404113195013626
  (0, 23)	0.0027381496483266897
  (0, 24)	-0.13747931304575628
  :	:
  (12091, 30)	0.2124760607764646
  (12091, 31)	0.15887535507808836
  (12091, 32)	0.03295798827909129
  (12091, 33)	-0.009067790056766046
  (12091, 34)	0.04197375444736147
  (12091, 35)	0.010435817339497905
  (12091, 36)	-0.032962810291187

## Construcción de la Matriz de Similitud

Se genera una representación vectorial combinando las características anteriores y se calcula la similitud coseno entre productos.

La función `cosine_similarity` se utiliza para medir la similitud entre vectores calculando el coseno del ángulo que forman en el espacio vectorial. Esta medida es especialmente útil en aplicaciones donde la dirección de los vectores es más relevante que su magnitud, como en sistemas de recomendación y procesamiento de lenguaje natural.

## Definición Matemática

Dados dos vectores $ \mathbf{a} $ y $ \mathbf{b} $, la similitud coseno se define como:

$$
\begin{equation}
\text{cosine\_similarity}(\mathbf{a}, \mathbf{b}) = \frac{\mathbf{a} \cdot \mathbf{b}}{\|\mathbf{a}\| \, \|\mathbf{b}\|}
\end{equation}
$$

donde:

- $ \mathbf{a} \cdot \mathbf{b} $ es el **producto escalar** de $ \mathbf{a} $ y $ \mathbf{b} $.
- $ \|\mathbf{a}\| $ es la **norma** del vector $ \mathbf{a} $, calculada como:

$$
\begin{equation}
\|\mathbf{a}\| = \sqrt{\sum_{i=1}^{n} a_i^2}
\end{equation}
$$

De manera similar, la norma del vector $ \mathbf{b} $ se calcula como:

$$
\begin{equation}
\|\mathbf{b}\| = \sqrt{\sum_{i=1}^{n} b_i^2}
\end{equation}
$$


El valor resultante de la similitud coseno se encuentra en el rango \([-1, 1]\), donde:

- **1** indica que los vectores son idénticos en dirección.
- **0** indica que los vectores son ortogonales (sin correlación en la dirección).
- **-1** indica que los vectores tienen direcciones opuestas.

## Uso en Scikit-Learn

En la librería [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html), se puede observar la documentación sobre la función `cosine_similarity`.

In [11]:
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

# Convertir a formato disperso
final_matrix_sparse = csr_matrix(final_matrix)

# Calcular similitud por bloques en memoria reducida
similarity_matrix = cosine_similarity(final_matrix_sparse, dense_output=False)


Resultados al aplicar al aplicar `cosine_similarity` para obtener la matriz de similaridad:

In [23]:
print(similarity_matrix)

[[1.         0.80634022 0.82877353 ... 0.76835073 0.70965516 0.76031432]
 [0.80634022 1.         0.83222845 ... 0.8456123  0.78819991 0.84106602]
 [0.82877353 0.83222845 1.         ... 0.85579312 0.80577697 0.851106  ]
 ...
 [0.76835073 0.8456123  0.85579312 ... 1.         0.89473971 0.93330927]
 [0.70965516 0.78819991 0.80577697 ... 0.89473971 1.         0.9309524 ]
 [0.76031432 0.84106602 0.851106   ... 0.93330927 0.9309524  1.        ]]


<p align="center"><strong>Tabla 1:</strong> Resultados Matriz de Similaridad.</p>

<p align="center"><em style="font-size: smaller;">Elaboración propia.</em></p>

In [13]:
product = "Allen Solly Men's Regular Fit Polo"

df[df['name'] == product ]

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,Allen Solly Men's Regular Fit Polo,men's clothing,T-shirts & Polos,https://m.media-amazon.com/images/I/71eUwDk8z+...,https://www.amazon.in/Allen-Solly-Regular-AMKP...,4.1,60020,590,1099
18,Allen Solly Men's Regular Fit Polo,men's clothing,T-shirts & Polos,https://m.media-amazon.com/images/I/81RcNGzlIh...,https://www.amazon.in/Allen-Solly-Regular-T-Sh...,4.2,3868,579,1199
43,Allen Solly Men's Regular Fit Polo,men's clothing,T-shirts & Polos,https://m.media-amazon.com/images/I/91tOI54L+n...,https://www.amazon.in/Allen-Solly-Regular-T-Sh...,4.2,3582,689,1149
1310,Allen Solly Men's Regular Fit Polo,men's clothing,T-shirts & Polos,https://m.media-amazon.com/images/I/61SFP-eFJp...,https://www.amazon.in/Allen-Solly-Solid-Regula...,5.0,1,689,1049
1686,Allen Solly Men's Regular Fit Polo,men's clothing,T-shirts & Polos,https://m.media-amazon.com/images/I/61bJZ+ZZil...,https://www.amazon.in/Allen-Solly-Solid-Regula...,3.9,12,839,1049
1830,Allen Solly Men's Regular Fit Polo,men's clothing,T-shirts & Polos,https://m.media-amazon.com/images/I/61f7aMUn5c...,https://www.amazon.in/Allen-Solly-Solid-Regula...,4.6,2,828,1049
1848,Allen Solly Men's Regular Fit Polo,men's clothing,T-shirts & Polos,https://m.media-amazon.com/images/I/61SFP-eFJp...,https://www.amazon.in/Allen-Solly-Solid-Regula...,5.0,1,689,1049
2490,Allen Solly Men's Regular Fit Polo,men's clothing,T-shirts & Polos,https://m.media-amazon.com/images/I/710Lu8wzRa...,https://www.amazon.in/Allen-Solly-Solid-Regula...,3.6,18,799,999
4111,Allen Solly Men's Regular Fit Polo,men's clothing,T-shirts & Polos,https://m.media-amazon.com/images/I/61qGNWBC1X...,https://www.amazon.in/Allen-Solly-Printed-Regu...,3.0,1,799,999
7666,Allen Solly Men's Regular Fit Polo,men's clothing,T-shirts & Polos,https://m.media-amazon.com/images/I/614ocjm7Xl...,https://www.amazon.in/Allen-Solly-Solid-Regula...,3.5,9,689,1049


<p align="center"><strong>Tabla 2:</strong> Top 10 de Productos.</p>

<p align="center"><em style="font-size: smaller;">Elaboración propia.</em></p>

In [14]:
# test = df.loc[950,'name']
# test

# Print the products in the cart by their indices
cart_indices = [0]

# Display cart items
cart_items = df.iloc[cart_indices][['name', 'main_category', 'actual_price', 'ratings', 'no_of_ratings']]
cart_items

,name,main_category,actual_price,ratings,no_of_ratings
0,Allen Solly Men's Regular Fit Polo,men's clothing,1099,4.1,60020


## Algoritmo de Recomendación

Dado un conjunto de productos seleccionados, el sistema:

1. Obtiene la matriz de similitud coseno.

2. Calcula el promedio de similitud con los productos en el carrito.

3. Ordena los productos según su similitud y filtra los más relevantes.

In [15]:
from scipy.sparse import csr_matrix
import numpy as np

def recommend(cart_indices, df, similarity_matrix, top_n=5):
    # Convertir a matriz densa si es necesario
    if isinstance(similarity_matrix, csr_matrix):
        similarity_matrix = similarity_matrix.toarray() 

    # Obtener puntajes de similitud para cada producto en el carrito
    sim_scores = np.mean([similarity_matrix[idx] for idx in cart_indices], axis=0)

    # Enumerar y ordenar por similitud descendente
    sim_scores = list(enumerate(sim_scores))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Remover los productos que ya están en el carrito
    sim_scores = [item for item in sim_scores if item[0] not in cart_indices]
    
    # Seleccionar los top_n índices, eliminando duplicados basados en 'name'
    unique_top_indices = []
    seen_names = set()
    for idx, score in sim_scores:
        product_name = df.iloc[idx]['name']
        if product_name not in seen_names:
            unique_top_indices.append(idx)
            seen_names.add(product_name)
        if len(unique_top_indices) >= top_n:
            break

    # Retornar detalles de los productos recomendados
    return df.iloc[unique_top_indices][['name', 'main_category', 'image', 'sub_category', 'actual_price', 'ratings', 'no_of_ratings']]


Se ejecuta el algoritmo de recomendación utilizando el primer producto del dataset, que corresponde a `Allen Solly Men's Regular Fit Polo`. A continuación, se presentan los resultados del algoritmo, que muestra los 10 productos más similares a este primer producto.

In [16]:
# Prueba con los índices del carrito
recommend(cart_indices, df, similarity_matrix, top_n=10)

,name,main_category,image,sub_category,actual_price,ratings,no_of_ratings
18,Allen Solly Men's Regular Fit Polo,men's clothing,https://m.media-amazon.com/images/I/81RcNGzlIh...,T-shirts & Polos,1199,4.2,3868
26,Allen Solly Men's Regular Fit T-Shirt,men's clothing,https://m.media-amazon.com/images/I/71m0PW9bIM...,T-shirts & Polos,799,4.0,3607
18025,Allen Solly Men's Regular Fit Polo Shirt,men's clothing,https://m.media-amazon.com/images/I/51LmPsR0XZ...,T-shirts & Polos,1499,4.0,1
14114,Allen Solly Men's Regular Polo Shirt,men's clothing,https://m.media-amazon.com/images/I/519jb9GGiQ...,T-shirts & Polos,1799,4.5,2
4613,Allen Solly Men's Regular Fit T-Shirt (ALKCVSG...,men's clothing,https://m.media-amazon.com/images/I/61KrpVDw9i...,T-shirts & Polos,799,4.3,10
4920,Allen Solly Men's Regular Fit T-Shirt (ALKCVSG...,men's clothing,https://m.media-amazon.com/images/I/61qj+dkW4U...,T-shirts & Polos,799,4.0,15
18275,Allen Solly Men's Regular Fit T-Shirt (ALKCVSG...,men's clothing,https://m.media-amazon.com/images/W/IMAGERENDE...,T-shirts & Polos,799,4.4,4
162,Allen Solly Men Polo,men's clothing,https://m.media-amazon.com/images/I/61XzgOm2z4...,T-shirts & Polos,999,4.2,38
8996,Allen Solly Men's Regular Fit T-Shirt (ALKCVSG...,men's clothing,https://m.media-amazon.com/images/I/61YOqs0tAI...,T-shirts & Polos,799,3.6,7
6287,Allen Solly Men's Regular Fit T-Shirt (ALKCVSG...,men's clothing,https://m.media-amazon.com/images/W/IMAGERENDE...,T-shirts & Polos,799,3.5,10


In [17]:
from IPython.display import display, HTML

def display_recommendations(cart_indices, df, similarity_matrix, top_n=5):
    recommendations = recommend(cart_indices, df, similarity_matrix, top_n)
    
    html = "<h3>Productos Recomendados</h3><table>"
    for _, row in recommendations.iterrows():
        html += f"""
        <tr>
            <td><img src="{row['image']}" width="100"></td>
            <td><b>{row['name']}</b><br>
                Categoría: {row['main_category']}<br>
                Subcategoría: {row['sub_category']}<br>
                Precio: {row['actual_price']}<br>
                Calificación: {row['ratings']} ({row['no_of_ratings']} valoraciones)
            </td>
        </tr>
        """
    html += "</table>"
    
    display(HTML(html))

# Llamada a la función
display_recommendations(cart_indices, df, similarity_matrix, top_n=10)


,Allen Solly Men's Regular Fit Polo Categoría: men's clothing Subcategoría: T-shirts & Polos Precio: 1199 Calificación: 4.2 (3868 valoraciones)
,Allen Solly Men's Regular Fit T-Shirt Categoría: men's clothing Subcategoría: T-shirts & Polos Precio: 799 Calificación: 4.0 (3607 valoraciones)
,Allen Solly Men's Regular Fit Polo Shirt Categoría: men's clothing Subcategoría: T-shirts & Polos Precio: 1499 Calificación: 4.0 (1 valoraciones)
,Allen Solly Men's Regular Polo Shirt Categoría: men's clothing Subcategoría: T-shirts & Polos Precio: 1799 Calificación: 4.5 (2 valoraciones)
,Allen Solly Men's Regular Fit T-Shirt (ALKCVSGFC84398_Navy_Medium) Categoría: men's clothing Subcategoría: T-shirts & Polos Precio: 799 Calificación: 4.3 (10 valoraciones)
,Allen Solly Men's Regular Fit T-Shirt (ALKCVSGF195222_Peach_Medium) Categoría: men's clothing Subcategoría: T-shirts & Polos Precio: 799 Calificación: 4.0 (15 valoraciones)
,Allen Solly Men's Regular Fit T-Shirt (ALKCVSGFE60941_Yellow Ochre_Small) Categoría: men's clothing Subcategoría: T-shirts & Polos Precio: 799 Calificación: 4.4 (4 valoraciones)
,Allen Solly Men Polo Categoría: men's clothing Subcategoría: T-shirts & Polos Precio: 999 Calificación: 4.2 (38 valoraciones)
,Allen Solly Men's Regular Fit T-Shirt (ALKCVSGFE60941_Yellow Ochre_Extra Large) Categoría: men's clothing Subcategoría: T-shirts & Polos Precio: 799 Calificación: 3.6 (7 valoraciones)
,Allen Solly Men's Regular Fit T-Shirt (ALKCVSGFC84398_Navy_Extra Large) Categoría: men's clothing Subcategoría: T-shirts & Polos Precio: 799 Calificación: 3.5 (10 valoraciones)


<p align="center"><strong>Figura 1:</strong> Productos Recomendados.</p>

<p align="center"><em style="font-size: smaller;">Elaboración propia.</em></p>

In [18]:
print(similarity_matrix.shape)

(12092, 12092)


## Matriz de Similitud entre Productos

In [19]:
similarity_matrix = similarity_matrix.toarray()

similarity_df = pd.DataFrame(similarity_matrix, index=df['name'], columns=df['name'])
print(similarity_df)

name                                                Allen Solly Men's Regular Fit Polo  \
name                                                                                     
Allen Solly Men's Regular Fit Polo                                            1.000000   
Van Heusen Men's Regular Fit Polo Shirt                                       0.806340   
Scott International Men's Regular Fit T-Shirt (...                            0.828774   
TOPLOT T-Shirt for Men||T Shirt||White T-Shirt ...                            0.724706   
Amazon Brand - Symbol Men's Regular Polo Shirt                                0.781704   
...                                                                                ...   
Puma Men's Polyester Sweatshirt                                               0.730426   
Ruggers Men's Striped Slim fit T-Shirt (2815316...                            0.750562   
Indian Terrain Men's Plain Slim Fit Polo (ITMTS...                            0.768351   
Workshop G

<p align="center"><strong>Tabla 3:</strong> Matriz de similitud entre Productos.</p>

<p align="center"><em style="font-size: smaller;">Elaboración propia.</em></p>

### Evaluación

Para evaluar el rendimiento del sistema se usan las siguientes métricas:

- **Precisión**: Fracción de recomendaciones relevantes sobre el total de recomendaciones realizadas.
- **Recall**: Fracción de productos relevantes correctamente recomendados.
- **F1-score**: Media armónica entre precisión y recall.

Resultados obtenidos:

$$
\begin{equation}
\text{Precision} = \frac{TP}{TP + FP}
\end{equation}
$$

$$
\begin{equation}
\text{Recall} = \frac{TP}{TP + FN}
\end{equation}
$$

$$
\begin{equation}
F1 = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
\end{equation}
$$

Donde **TP** son los productos relevantes recomendados, **FP** son las recomendaciones incorrectas y **FN** son los productos relevantes no recomendados.

Con los resultados obtenidos:

- **Precisión = 0.60**: Indica que el 60% de los productos recomendados fueron realmente relevantes para el usuario.
- **Recall = 1.00**: Demuestra que se lograron recuperar todos los productos relevantes del conjunto de datos.
- **F1-score = 0.75**: Representa el balance entre precisión y recall, con un buen desempeño general.

El alto valor de **recall** muestra que el sistema es efectivo en identificar productos relevantes, asegurando que los usuarios vean todas las opciones adecuadas.



In [20]:
# Supongamos que estos son los productos relevantes para el usuario (ground truth)
relevant_items = {"Allen Solly Men's Regular Fit Polo", "Allen Solly Men's Regular Fit Polo Shirt", "Allen Solly Men's Regular Fit T-Shirt (ALKCVSGFC84398_Navy_Medium)"}

# Y estos son los productos que el sistema recomendó
recommended_items = {"Adidas Men T-Shirt", "Allen Solly Men's Snug Fit T-Shirt", "Allen Solly Men's Regular Fit T-Shirt (ALKCVSGFE60941_Yellow Ochre_Extra Large)", "Allen Solly Men's Regular Fit Polo", "Allen Solly Men's Regular Fit T-Shirt", "Allen Solly Men's Regular Fit Polo Shirt",  "Allen Solly Men Polo"}

def precision_recall_f1(relevant_items, recommended_items):
    # Intersección entre relevantes y recomendados
    true_positives = len(relevant_items.intersection(recommended_items))
    
    # Precision: cuántos recomendados son realmente relevantes
    precision = true_positives / len(recommended_items) if recommended_items else 0
    
    # Recall: cuántos relevantes fueron recomendados
    recall = true_positives / len(relevant_items) if relevant_items else 0
    
    # F1-score: balance entre precisión y recall
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0

    return precision, recall, f1

recommended_df = recommend(cart_indices, df, similarity_matrix, top_n=5)

# Convertir los nombres de los productos recomendados en un conjunto
recommended_items = set(recommended_df['name'])  

# Evaluación de precisión y recall
precision, recall, f1 = precision_recall_f1(relevant_items, recommended_items)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")


Precision: 0.60
Recall: 1.00
F1-score: 0.75


In [21]:
def get_top_similarities(similarity_df, product_name, top_n=10):
    # Verifica que el producto exista en el DataFrame
    if product_name not in similarity_df.index:
        print("Producto no encontrado")
        return None
    
    # Obtener la fila correspondiente al producto
    sim_data = similarity_df.loc[product_name]
    
    # Si sim_data es un DataFrame (por índices duplicados), tomar la primera fila
    if isinstance(sim_data, pd.DataFrame):
        sim_series = sim_data.iloc[0]
    else:
        sim_series = sim_data
    
    # Ordenar la Serie de forma descendente
    sorted_sim = sim_series.sort_values(ascending=False)
    
    # Excluir el propio producto (donde la similitud es 1.0)
    sorted_sim = sorted_sim[sorted_sim.index != product_name]
    
    # Devolver los top_n valores
    return sorted_sim.head(top_n)

# Ejemplo de uso:
product_name = "Allen Solly Men's Regular Fit Polo"
top_similarities = get_top_similarities(similarity_df, product_name, top_n=60)
print(top_similarities)


name
Allen Solly Men's Regular Fit T-Shirt                                              0.889098
Allen Solly Men's Regular Fit Polo Shirt                                           0.885141
Allen Solly Men's Regular Fit Polo Shirt                                           0.885112
Allen Solly Men's Regular Fit Polo Shirt                                           0.885067
Allen Solly Men's Regular Fit Polo Shirt                                           0.884802
Allen Solly Men's Regular Fit Polo Shirt                                           0.884726
Allen Solly Men's Regular Fit Polo Shirt                                           0.884726
Allen Solly Men's Regular Fit Polo Shirt                                           0.884726
Allen Solly Men's Regular Fit Polo Shirt                                           0.884539
Allen Solly Men's Regular Fit Polo Shirt                                           0.884539
Allen Solly Men's Regular Fit Polo Shirt                                   

## Conclusiones

- El sistema de recomendación basado en similitud de contenido es capaz de generar sugerencias relevantes para los usuarios, aprovechando características textuales y numéricas de los productos. Su buen desempeño en recall garantiza que los productos adecuados sean considerados, mejorando la experiencia de búsqueda y exploración en plataformas de comercio electrónico.

- Al trabajar con matrices diversas, hay que tener en cuenta la dimensionalidad y la eficiencia computacional. La representación dispersa de matrices, como en el caso de TF-IDF o embeddings de productos, puede ser útil para reducir el consumo de memoria y acelerar cálculos de similitud. Es fundamental elegir técnicas de reducción de dimensionalidad adecuadas, como Truncated SVD o PCA, para evitar la pérdida de información clave y mejorar el rendimiento del modelo sin comprometer la precisión de las recomendaciones.

## Referencias

Scikit-learn. (s.f.). Cosine similarity. Recuperado el 18 de febrero de 2025, de https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html

GeeksforGeeks. (s.f.). Cosine similarity. Recuperado el 18 de febrero de 2025, de https://www.geeksforgeeks.org/cosine-similarity/

Scikit-learn. (s.f.). Precision score. Recuperado el 18 de febrero de 2025, de https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html

GeeksforGeeks. (s.f.). Understanding TF-IDF (Term Frequency - Inverse Document Frequency). Recuperado el 18 de febrero de 2025, de https://www.geeksforgeeks.org/understanding-tf-idf-term-frequency-inverse-document-frequency/

Scikit-learn. (s.f.). Truncated SVD. Recuperado el 18 de febrero de 2025, de https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4ce25df3-b939-4195-8b6c-572825e9af1f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>